### Quantum Fisher Information matrix

<img src="../images/qfim.png">

I have coded QFIM function and test it on 1qubit case

$\partial_\theta|\psi\rangle$ can be calculated by on the below equation (from Le Bin Ho):

<img src="../images/grad_psi.png" width=500px/>

The QNG are not simple as our imagin, the det of QFIM is 0 at almost step, so when using Moore-Penrose pseudo inverse the results're not good. The new solution is based on [this paper](https://arxiv.org/pdf/1909.02108.pdf). It seems to more complex, there are many that need to code:

- Create function that return the generator for every circuit

- Calculating the Fubini-Study tensor

- Divide circuit into some layer, and use recursion on it.

$R_X(\theta)=e^{i\frac{\theta}{2}X}$

In [1]:
import qiskit
import numpy as np
import qtm.base_qtm, qtm.constant, qtm.qtm_1qubit, qtm.qtm_nqubit
import matplotlib.pyplot as plt
from types import FunctionType

In [2]:
def create_psi2(thetas):
    qc = qiskit.QuantumCircuit(3, 3)
    # |psi_0>: state preparation
    qc.ry(np.pi / 4, 0)
    qc.ry(np.pi / 3, 1)
    qc.ry(np.pi / 7, 2)
    # V0(theta0, theta1): Parametrized layer 0
    qc.rz(thetas[0], 0)
    qc.rz(thetas[1], 1)
    # W1: non-parametrized gates
    qc.cnot(0, 1)
    qc.cnot(1, 2)
    return qc

def create_psi():
    qc = qiskit.QuantumCircuit(3, 3)
    qc.ry(np.pi / 4, 0)
    qc.ry(np.pi / 3, 1)
    qc.ry(np.pi / 7, 2)
    return qc


In [11]:
qc = create_psi()
from typing import Dict
def calculate_fubini_study(qc, observers: Dict[str, int]):
    # Get |psi>
    psi = qiskit.quantum_info.Statevector.from_instruction(qc).data
    psi = np.expand_dims(psi , 1)
    # Get <psi|
    psi_hat = np.transpose(np.conjugate(psi))
    num_observers = len(observers)
    num_qubits = qc.num_qubits
    g = np.zeros([num_observers, num_observers], dtype=np.complex128) 
    # Each K[j] must have 2^n x 2^n dimensional with n is the number of qubits   
    Ks = []

    for observer_name, observer_wire in observers:
        observer = qtm.constant.generator[observer_name]
        if observer_wire == 0:
            K = observer
        else:
            K = qtm.constant.generator['I']
        for i in range(1, num_qubits):
            if i == observer_wire:
                K = np.kron(K, observer)
            else:
                K = np.kron(K, qtm.constant.generator['I'])
        
        Ks.append(K)

    for i in range(0, num_observers):
        for j in range(0, num_observers):
            g[i, j] = psi_hat @ (Ks[i] @ Ks[j]) @ psi - (psi_hat @ Ks[i] @ psi)*(psi_hat @ Ks[j] @ psi)
            if g[i, j] < 10**(-10):
                g[i, j] = 0
    return g

observers = [
    ['RZ', 2],
    ['RZ', 1]
]

calculate_fubini_study(qc, observers)

array([[0.125 +0.j, 0.    +0.j],
       [0.    +0.j, 0.1875+0.j]])

In [50]:
def m(qc, x = [], y = [], z = []):
    qc = qtm.base_qtm.x_measurement(qc, qubits = x)
    qc = qtm.base_qtm.y_measurement(qc, qubits = y)
    qc = qtm.base_qtm.z_measurement(qc, qubits = z)
    return qc
# calculate g_ij
qc = create_psi()
xs, ys, zs = [], [], [0, 1]
qc = m(qc, z = zs)

counts = qiskit.execute(
    qc, 
    backend = qtm.constant.backend, 
    shots = qtm.constant.num_shots
).result().get_counts()


for i in counts:
    counts[i] /= qtm.constant.num_shots

p_single = np.zeros([len(xs) + len(ys) + len(zs), 2])

for i in range(0, len(zs)):
    p_single[zs[i], 0] = sum([v for k, v in counts.items() if k[qc.num_qubits-1-zs[i]] == '0'])
    p_single[zs[i], 1] = sum([v for k, v in counts.items() if k[qc.num_qubits-1-zs[i]] == '1'])

p_double = np.zeros([])

p_q0q1_00 = sum([v for k, v in counts.items() if k[-2:] == '00'])
p_q0q1_01 = sum([v for k, v in counts.items() if k[-2:] == '10'])
p_q0q1_10 = sum([v for k, v in counts.items() if k[-2:] == '01'])
p_q0q1_11 = sum([v for k, v in counts.items() if k[-2:] == '11'])


z0 = p_single[0, 0] - p_single[0, 1]

print('z0', z0)
z0_2 = 1
z1 = p_single[1, 0] - p_single[1, 1]
z1_2 = 1

z0_z1 = (p_single[0, 0] - p_single[0, 1])*(p_single[1, 0] - p_single[1, 1])
z0z1 = p_q0q1_00 - p_q0q1_01 - p_q0q1_10 + p_q0q1_11

g0 = np.zeros([2, 2])
g0[0, 0] = z0_2 - z0**2
g0[0, 1] = z0z1 - z0_z1
g0[1, 0] = z0z1 - z0_z1
g0[1, 1] = z1_2 - z1**2

qc.draw('mpl')

print(g0 / 4)


z0 0.7041999999999999
[[ 0.12602559 -0.00077987]
 [-0.00077987  0.18512791]]


$|\psi_0\rangle=R_Y(\pi/4)\otimes R_Y(\pi/3)\otimes R_Y(\pi/7)$

So $|\psi\rangle$ in $p(ij)$ formula means $R_Y(\pi/4)\otimes R_Y(\pi/3)$?

In [5]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=3)
g0 = np.zeros([2, 2])
params = np.array([0.432, -0.123, 0.543, 0.233])

def layer0_subcircuit(params):
    """This function contains all gates that
    precede parametrized layer 0"""
    qml.RY(np.pi / 4, wires=0)
    qml.RY(np.pi / 3, wires=1)
    qml.RY(np.pi / 7, wires=2)
@qml.qnode(dev)
def layer0_diag(params):
    layer0_subcircuit(params)
    return qml.var(qml.PauliZ(0)), qml.var(qml.PauliZ(1))


# calculate the diagonal terms
varK0, varK1 = layer0_diag(params)
g0[0, 0] = varK0 / 4
g0[1, 1] = varK1 / 4

print(g0)

[[0.125  0.    ]
 [0.     0.1875]]


In [2]:
g1 = np.zeros([2, 2])


def layer1_subcircuit(params):
    """This function contains all gates that
    precede parametrized layer 1"""
    # |psi_0>: state preparation
    qml.RY(np.pi / 4, wires=0)
    qml.RY(np.pi / 3, wires=1)
    qml.RY(np.pi / 7, wires=2)

    # V0(theta0, theta1): Parametrized layer 0
    qml.RZ(params[0], wires=0)
    qml.RZ(params[1], wires=1)

    # W1: non-parametrized gates
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])

In [3]:
@qml.qnode(dev)
def layer1_diag(params):
    layer1_subcircuit(params)
    return qml.var(qml.PauliY(1)), qml.var(qml.PauliX(2))

In [5]:
varK0, varK1 = layer1_diag(params)
g1[0, 0] = varK0 / 4
g1[1, 1] = varK1 / 4
@qml.qnode(dev)
def layer1_off_diag_single(params):
    layer1_subcircuit(params)
    return qml.expval(qml.PauliY(1)), qml.expval(qml.PauliX(2))


@qml.qnode(dev)
def layer1_off_diag_double(params):
    layer1_subcircuit(params)
    X = np.array([[0, 1], [1, 0]])
    Y = np.array([[0, -1j], [1j, 0]])
    YX = np.kron(Y, X)
    return qml.expval(qml.Hermitian(YX, wires=[1, 2]))


# calculate the off-diagonal terms
exK0, exK1 = layer1_off_diag_single(params)
exK0K1 = layer1_off_diag_double(params)

g1[0, 1] = (exK0K1 - exK0 * exK1) / 4
g1[1, 0] = g1[0, 1]

In [6]:
g1

tensor([[ 0.24973433, -0.01524701],
        [-0.01524701,  0.20293623]], requires_grad=True)

In [21]:



thetas = np.array([0.432, -0.123, 0.543, 0.233])
# calculate g_ij
qc = create_psi2(thetas)
qc = y_measurement(qc, 1, 1)
qc = x_measurement(qc, 2, 2)


counts = qiskit.execute(qc, backend = qtm.constant.backend, shots = qtm.constant.num_shots).result().get_counts()

for i in counts:
    counts[i] /= qtm.constant.num_shots

print(counts)

p_q0_0 = sum([v for k, v in counts.items() if k[-2] == '0'])
p_q0_1 = sum([v for k, v in counts.items() if k[-2] == '1'])
p_q1_0 = sum([v for k, v in counts.items() if k[-3] == '0'])
p_q1_1 = sum([v for k, v in counts.items() if k[-3] == '1'])

p_q0q1_00 = sum([v for k, v in counts.items() if k[-3:-1] == '00'])
p_q0q1_01 = sum([v for k, v in counts.items() if k[-3:-1] == '10'])
p_q0q1_10 = sum([v for k, v in counts.items() if k[-3:-1] == '01'])
p_q0q1_11 = sum([v for k, v in counts.items() if k[-3:-1] == '11'])


y1 = p_q0_0 - p_q0_1
y1_2 = 1
x2 = p_q1_0 - p_q1_1
x2_2 = 1

y1_x2 = (p_q0_0 - p_q0_1)*(p_q1_0 - p_q1_1)
y1x2 = p_q0q1_00 - p_q0q1_01 - p_q0q1_10 + p_q0q1_11

g1 = np.zeros([2, 2])
g1[0, 0] = y1_2 - y1**2
g1[0, 1] = y1x2 - y1_x2
g1[1, 0] = y1x2 - y1_x2
g1[1, 1] = x2_2 - x2**2
print(g1/4)

{'000': 0.3256, '010': 0.3929, '110': 0.1278, '100': 0.1537}
[[ 0.24957151 -0.01877705]
 [-0.01877705  0.20225775]]
